In [1]:
# 在 Notebook 裡跑
import sys
!{sys.executable} -m pip install -U ipywidgets jupyterlab_widgets widgetsnbextension

In [2]:
import ipywidgets, jupyterlab_widgets, widgetsnbextension
print("ipywidgets:", ipywidgets.__version__)
print("jupyterlab_widgets:", jupyterlab_widgets.__version__)
print("widgetsnbextension:", widgetsnbextension.__version__)

ipywidgets: 8.1.7
jupyterlab_widgets: 3.0.15
widgetsnbextension: 4.0.14


In [3]:
import ipywidgets as W
from IPython.display import display

display(W.HTML("<b>ipywidgets OK</b>"))
display(W.IntSlider(value=3, min=0, max=10, description="測試滑桿"))

HTML(value='<b>ipywidgets OK</b>')

IntSlider(value=3, description='測試滑桿', max=10)

In [4]:
# --- 麥當勞點餐機 (ipywidgets) — PNG / 圖片在中欄 / 結帳後產生收據 / 高品質顯示 ---
# pip install ipywidgets pillow

from IPython.display import display
import ipywidgets as W
from datetime import datetime
import re, os, io
from PIL import Image

# ========= 資料 =========
PRICES_MAIN = {"bigmac": 89, "mcchicken": 79, "filetofish": 95, "dblcheese": 99}
PRICES_FRIES = {"fries_s": 35, "fries_m": 45, "fries_l": 55}
PRICES_NUGGETS = {"nuggets4": 49, "nuggets6": 65, "nuggets10": 109}
PRICES_SNACK_OTHER = {"apple_pie": 42, "corn_soup": 55, "side_salad": 50, "sundae_choco": 45}
PRICES_DRINK = {"coke": 38, "sprite": 36, "iced_tea": 32, "hot_coffee": 40}

NAME_MAIN = {"bigmac": "大麥克", "mcchicken": "麥香雞", "filetofish": "麥香魚", "dblcheese": "雙層牛肉吉事堡"}
NAME_FRIES_SIZE = {"fries_s": "小薯", "fries_m": "中薯", "fries_l": "大薯"}
NAME_FRIES_SALT = {"nosalt": "無鹽", "salt": "有鹽"}
NAME_NUGGETS_SIZE = {"nuggets4": "4塊", "nuggets6": "6塊", "nuggets10": "10塊"}
NAME_NUGGETS_SAUCE = {"none": "不需要醬料", "sweet_chili": "甜辣醬", "mustard": "芥末醬", "bbq": "BBQ醬"}
NAME_SNACK_OTHER = {"apple_pie": "蘋果派", "corn_soup": "玉米濃湯", "side_salad": "田園沙拉", "sundae_choco": "巧克力聖代"}
NAME_DRINK = {"coke": "可口可樂(去冰)", "sprite": "雪碧", "iced_tea": "冰紅茶", "hot_coffee": "熱美式"}

# 圖片（PNG）
IMG_MAP = {
    "bigmac": "bigmac.png", "mcchicken": "mcchicken.png", "filetofish": "filetofish.png", "dblcheese": "dblcheese.png",
    "fries_s": "fries_s.png", "fries_m": "fries_m.png", "fries_l": "fries_l.png",
    "nuggets4": "nuggets4.png", "nuggets6": "nuggets6.png", "nuggets10": "nuggets10.png",
    "apple_pie": "apple_pie.png", "corn_soup": "corn_soup.png", "side_salad": "side_salad.png", "sundae_choco": "sundae_choco.png",
    "coke": "coke.png", "sprite": "sprite.png", "iced_tea": "iced_tea.png", "hot_coffee": "hot_coffee.png",
}
ASSETS_DIR = "./assets"
DEFAULT_IMG = "none.png"

ORDER_COUNTER = {"n": 0}
CART = {}  # sku -> {name_display, unit_price, qty, category}

# ========= 圖片：只縮不放 & 高品質 =========
def load_image_for_widget(key: str, max_w=380, max_h=300):
    """回傳 (bytes, w, h)。若超出上限則等比例縮小；小圖不放大。"""
    filename = IMG_MAP.get(key, DEFAULT_IMG)
    path = os.path.join(ASSETS_DIR, filename)
    if not os.path.exists(path):
        path = os.path.join(ASSETS_DIR, DEFAULT_IMG)
    try:
        with Image.open(path) as im:
            im = im.convert("RGBA")
            ow, oh = im.size
            scale = min(max_w / ow, max_h / oh, 1.0)  # 只縮不放
            if scale < 1.0:
                nw, nh = int(ow * scale), int(oh * scale)
                im = im.resize((nw, nh), resample=Image.LANCZOS)
            else:
                nw, nh = ow, oh
            bio = io.BytesIO()
            im.save(bio, format="PNG")
            return bio.getvalue(), nw, nh
    except Exception:
        return b"", 0, 0

def currency(n: int) -> str: return f"${n:,}"
def next_order_id() -> str:
    ORDER_COUNTER["n"] += 1
    return f"MCD-{datetime.now().strftime('%Y%m%d-%H%M%S')}-{ORDER_COUNTER['n']:03d}"
def tw_tax_id_ok(b: str) -> bool: return bool(re.fullmatch(r"\d{8}", b))
def carrier_hint_ok(c: str) -> bool: return bool(re.fullmatch(r"/[A-Z0-9.+-]{7}", c.upper()))

# ========= UI 基礎 =========
title = W.HTML("<h2>🍔 麥當勞點餐機（ipywidgets / PNG）</h2>")
tb_category = W.ToggleButtons(options=[("主餐","main"),("點心","snack"),("飲料","drink")], value="main", description="類別")
product_area = W.VBox()

# 圖片放中欄（先建空 Image，實際寬高由 set_preview 設）
img_preview = W.Image(format="png")
img_center = W.HBox([img_preview], layout=W.Layout(justify_content="center"))

# 右欄：明細/小計/備註/結帳
lbl_subtotal = W.HTML("<b>小計：$0</b>")
cart_area = W.VBox()
txt_note = W.Textarea(placeholder="請輸入備註（例如：飲料少冰、餐點不加洋蔥）", description="備註",
                      layout=W.Layout(width="100%", height="70px"))
cb_bag = W.Checkbox(value=False, description="需要購物袋 (+1)")
txt_tax_id = W.Text(value="", placeholder="8 碼數字，例如 12345678", description="統編")
txt_carrier = W.Text(value="", placeholder="/AB12345（手機條碼，可留白）", description="載具")
btn_checkout = W.Button(description="結帳", button_style="success", icon="credit-card")
out_receipt = W.Output()
checkout_panel = W.VBox([cb_bag, txt_tax_id, txt_carrier, btn_checkout, out_receipt])

# 三欄
left_col = W.VBox([tb_category])
mid_col  = W.VBox([product_area, W.HTML("<hr>"), W.HTML("<b>圖片預覽</b>"), img_center])
right_col = W.VBox([
    W.HTML("<b>購物車明細</b>"), cart_area, lbl_subtotal,
    W.HTML("<hr>"), txt_note,
    W.HTML("<hr>"), W.HTML("<b>結帳</b>"), checkout_panel
])
ui = W.HBox([left_col, mid_col, right_col],
            layout=W.Layout(width="100%", justify_content="space-between", align_items="flex-start", gap="24px"))

# ========= 購物車 =========
def compute_subtotal() -> int:
    return sum(i["unit_price"]*i["qty"] for i in CART.values()) + (1 if cb_bag.value else 0)
def update_subtotal_label(): lbl_subtotal.value = f"<b>小計：{currency(compute_subtotal())}</b>"
def remove_cart_item(sku: str):
    if sku in CART: del CART[sku]
    render_cart()
def on_qty_changed(change, sku: str):
    if change["name"]!="value": return
    try: newq = int(change["new"])
    except: newq = 0
    newq = max(0, newq)
    if newq==0: remove_cart_item(sku)
    else:
        CART[sku]["qty"]=newq
        render_cart()
def render_cart():
    rows=[]
    if not CART:
        rows.append(W.HTML("<i>購物車目前是空的</i>"))
    else:
        rows.append(W.HBox([
            W.HTML("<b>品項</b>", layout=W.Layout(width="55%")),
            W.HTML("<b>單價</b>", layout=W.Layout(width="12%")),
            W.HTML("<b>份數</b>", layout=W.Layout(width="16%")),
            W.HTML("<b>小計</b>", layout=W.Layout(width="12%")),
            W.HTML("", layout=W.Layout(width="5%")),
        ]))
        for sku,item in CART.items():
            name, unit, qty = item["name_display"], item["unit_price"], item["qty"]
            line = unit*qty
            name_w = W.HTML(name, layout=W.Layout(width="55%"))
            unit_w = W.Label(currency(unit), layout=W.Layout(width="12%"))
            qty_w  = W.IntText(value=qty, min=0, step=1, layout=W.Layout(width="90%"))
            qty_w.observe(lambda ch, s=sku: on_qty_changed(ch, s), names="value")
            sub_w  = W.Label(currency(line), layout=W.Layout(width="12%"))
            btn_del= W.Button(icon="trash", tooltip="刪除", layout=W.Layout(width="32px"))
            btn_del.on_click(lambda b, s=sku: remove_cart_item(s))
            rows.append(W.HBox([name_w, unit_w, W.HBox([qty_w], layout=W.Layout(width="16%")), sub_w, btn_del]))
    cart_area.children = rows
    update_subtotal_label()

# ========= 圖片預覽 =========
def set_preview(key: str):
    data, w, h = load_image_for_widget(key)  # 只縮不放
    img_preview.value = data
    if w and h:
        img_preview.width = w
        img_preview.height = h

# ========= 加入購物車 =========
def add_to_cart(sku: str, name_display: str, unit_price: int, qty: int, category: str):
    if qty<=0: return
    if sku in CART: CART[sku]["qty"] += qty
    else: CART[sku] = {"name_display": name_display, "unit_price": unit_price, "qty": qty, "category": category}
    render_cart()

# ========= 中欄商品卡 =========
status_msg = W.HTML("")

def build_main_card():
    dd_item = W.Dropdown(options=[(NAME_MAIN[k], k) for k in PRICES_MAIN], value="bigmac", description="主餐")
    qty     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus")
    status_msg.value = ""
    dd_item.observe(lambda ch: set_preview(dd_item.value), names="value")
    set_preview(dd_item.value)
    def _add(_):
        pid = dd_item.value
        add_to_cart(pid, NAME_MAIN[pid], PRICES_MAIN[pid], qty.value, "main")
        status_msg.value = f"<span style='color:green'>已加入：{NAME_MAIN[pid]} × {qty.value}</span>"
    btn_add.on_click(_add)
    return W.VBox([W.HBox([dd_item, qty, btn_add]), status_msg])

def build_snack_card():
    dd_kind = W.ToggleButtons(options=[("薯條","fries"),("麥克雞塊","nuggets"),("其他點心","other")], value="fries", description="點心類型")

    # 薯條
    dd_fries_size = W.Dropdown(options=[(NAME_FRIES_SIZE[k],k) for k in PRICES_FRIES], value="fries_m", description="尺寸")
    dd_fries_salt = W.Dropdown(options=[("無鹽","nosalt"),("有鹽","salt")], value="salt", description="口味")
    qty_fries     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_fries = W.Button(description="加入購物車", button_style="primary", icon="plus")
    dd_fries_size.observe(lambda ch: set_preview(dd_fries_size.value), names="value")
    set_preview(dd_fries_size.value)
    def add_fries(_):
        size, salt = dd_fries_size.value, dd_fries_salt.value
        name = f"{NAME_FRIES_SIZE[size]}（{NAME_FRIES_SALT[salt]}）"
        add_to_cart(f"{size}|{salt}", name, PRICES_FRIES[size], qty_fries.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty_fries.value}</span>"
    btn_add_fries.on_click(add_fries)
    fries_card = W.VBox([W.HBox([dd_fries_size, dd_fries_salt]), W.HBox([qty_fries, btn_add_fries])])

    # 麥克雞塊
    dd_n_size     = W.Dropdown(options=[(NAME_NUGGETS_SIZE[k],k) for k in PRICES_NUGGETS], value="nuggets6", description="份量")
    cb_need_sauce = W.Checkbox(value=True, description="需要醬料")
    dd_n_sauce    = W.Dropdown(options=[("不需要","none"),("甜辣醬","sweet_chili"),("芥末醬","mustard"),("BBQ醬","bbq")],
                               value="sweet_chili", description="醬料")
    qty_n         = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_n     = W.Button(description="加入購物車", button_style="primary", icon="plus")
    dd_n_size.observe(lambda ch: set_preview(dd_n_size.value), names="value")
    set_preview(dd_n_size.value)
    def on_cb_need(change):
        dd_n_sauce.disabled = not cb_need_sauce.value
        if not cb_need_sauce.value: dd_n_sauce.value = "none"
    cb_need_sauce.observe(on_cb_need, names="value"); on_cb_need(None)
    def add_n(_):
        nsize, sauce = dd_n_size.value, dd_n_sauce.value
        sauce_name = NAME_NUGGETS_SAUCE[sauce] if cb_need_sauce.value else NAME_NUGGETS_SAUCE["none"]
        name = f"麥克雞塊 {NAME_NUGGETS_SIZE[nsize]}（{sauce_name}）"
        add_to_cart(f"{nsize}|{sauce if cb_need_sauce.value else 'none'}", name, PRICES_NUGGETS[nsize], qty_n.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{name} × {qty_n.value}</span>"
    btn_add_n.on_click(add_n)
    nuggets_card = W.VBox([W.HBox([dd_n_size, cb_need_sauce, dd_n_sauce]), W.HBox([qty_n, btn_add_n])])

    # 其他點心
    dd_other      = W.Dropdown(options=[(NAME_SNACK_OTHER[k],k) for k in PRICES_SNACK_OTHER], value="apple_pie", description="品項")
    qty_other     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add_other = W.Button(description="加入購物車", button_style="primary", icon="plus")
    dd_other.observe(lambda ch: set_preview(dd_other.value), names="value")
    set_preview(dd_other.value)
    def add_other(_):
        pid = dd_other.value
        add_to_cart(pid, NAME_SNACK_OTHER[pid], PRICES_SNACK_OTHER[pid], qty_other.value, "snack")
        status_msg.value = f"<span style='color:green'>已加入：{NAME_SNACK_OTHER[pid]} × {qty_other.value}</span>"
    btn_add_other.on_click(add_other)
    other_card = W.VBox([W.HBox([dd_other]), W.HBox([qty_other, btn_add_other])])

    area = W.VBox()
    def rebuild_area(_=None):
        if dd_kind.value=="fries": area.children=[fries_card]; set_preview(dd_fries_size.value)
        elif dd_kind.value=="nuggets": area.children=[nuggets_card]; set_preview(dd_n_size.value)
        else: area.children=[other_card]; set_preview(dd_other.value)
    dd_kind.observe(rebuild_area, names="value"); rebuild_area()
    return W.VBox([dd_kind, area, status_msg])

def build_drink_card():
    dd_item = W.Dropdown(options=[(NAME_DRINK[k],k) for k in PRICES_DRINK], value="coke", description="飲料")
    qty     = W.IntText(value=1, min=1, step=1, description="份數")
    btn_add = W.Button(description="加入購物車", button_style="primary", icon="plus")
    status_msg.value = ""
    dd_item.observe(lambda ch: set_preview(dd_item.value), names="value")
    set_preview(dd_item.value)
    def _add(_):
        pid = dd_item.value
        add_to_cart(pid, NAME_DRINK[pid], PRICES_DRINK[pid], qty.value, "drink")
        status_msg.value = f"<span style='color:green'>已加入：{NAME_DRINK[pid]} × {qty.value}</span>"
    btn_add.on_click(_add)
    return W.VBox([W.HBox([dd_item, qty, btn_add]), status_msg])

def rebuild_product_area(_=None):
    if tb_category.value=="main": product_area.children=[build_main_card()]
    elif tb_category.value=="snack": product_area.children=[build_snack_card()]
    else: product_area.children=[build_drink_card()]
tb_category.observe(rebuild_product_area, names="value"); rebuild_product_area()

# ========= 結帳（按下才產生收據） =========
def generate_receipt(_):
    out_receipt.clear_output()
    order_id = next_order_id()
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    tax_id, carrier, note = txt_tax_id.value.strip(), txt_carrier.value.strip(), txt_note.value.strip()
    tax_line = "（未提供）" if not tax_id else tax_id + ("（統編有效）" if tw_tax_id_ok(tax_id) else "（格式需為 8 碼數字，仍可結帳）")
    carrier_line = "（未提供）" if not carrier else carrier + ("" if carrier_hint_ok(carrier) else "（載具格式與常見型式不符，仍可結帳）")

    lines=[]
    if not CART and not cb_bag.value: lines += ["(提示) 尚未選購任何品項",""]
    lines += [f"訂單編號：{order_id}", f"結帳時間：{ts}", f"統一編號：{tax_line}", f"載具：{carrier_line}", f"備註：{note if note else '（未提供）'}", "", "— 訂單明細 —"]
    for item in CART.values():
        name, unit, qty = item["name_display"], item["unit_price"], item["qty"]
        lines.append(f"- {name} × {qty}   {currency(unit)}" if qty==1 else f"- {name} × {qty}   {currency(unit)} × {qty} = {currency(unit*qty)}")
    if cb_bag.value: lines.append("購物袋：$1")
    subtotal_wo_bag = sum(i["unit_price"]*i["qty"] for i in CART.values())
    total = subtotal_wo_bag + (1 if cb_bag.value else 0)
    lines += ["--------------------------------", f"小計：{currency(subtotal_wo_bag)}", f"總計：{currency(total)}"]
    with out_receipt:
        print("\n".join(lines))

btn_checkout.on_click(generate_receipt)
cb_bag.observe(lambda ch: update_subtotal_label(), names="value")

# 初始渲染（預覽先載入大麥克）
set_preview("bigmac")
render_cart()
display(title, ui)

HTML(value='<h2>🍔 麥當勞點餐機（ipywidgets / PNG）</h2>')